In [1]:
using DrWatson
@quickactivate "Doran_etal_2022"
using SPI
using NewickTreeTools

In [2]:
# tiplabels = string.([:A, :B, :C, :D, :E])
dij = calc_spi_mtx(rand(100,50))
hc = hclust(dij, linkage=:average, branchorder=:optimal);

In [3]:
SPI.nwstr(hc);
NewickTree.nwstr(newick(hc));
@time SPI.nwstr(hc);
@time NewickTree.nwstr(newick(hc));

  0.000079 seconds (1.10 k allocations: 174.859 KiB)
  0.000455 seconds (4.88 k allocations: 539.500 KiB)


In [4]:
# tiplabels = string.([:A, :B, :C, :D, :E])
dij = calc_spi_mtx(rand(20000,50))
hc = hclust(dij, linkage=:average, branchorder=:optimal);

In [7]:
SPI.nwstr(hc);
NewickTree.nwstr(newick(hc));
@time SPI.nwstr(hc);
@time NewickTree.nwstr(newick(hc));

  0.021892 seconds (220.00 k allocations: 51.007 MiB, 18.02% gc time)
  0.179651 seconds (983.57 k allocations: 147.336 MiB, 24.87% gc time)
